In [1]:
import numpy as np
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense,Dropout,Convolution2D,MaxPooling2D,Flatten
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import skimage
from keras.layers import GlobalAveragePooling2D
from tensorflow.keras import layers
from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler
from tensorflow import keras
import spams
import pandas as pd

2023-06-11 20:19:18.547248: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:
#使用chtgpt给的模型加权

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [4]:
data = np.load('/home/xipeng/FLcode/organamnist.npz')
data.files

x_train = data["train_images"]
y_train = data["train_labels"]
x_test = data["test_images"]
y_test = data["test_labels"]

# 为图像数据添加通道维度
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

# 归一化图像数据
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0



import random

# 设备数量和IID程度
num_devices = 30
iid_ratio = 1

# 计算每个设备应包含的类别数量
num_classes = len(np.unique(y_train))
samples_per_device = len(x_train) // num_devices
selected_classes_per_device = int(num_classes * iid_ratio)

# 分配数据
device_data = [[] for _ in range(num_devices)]

# 为每个设备分配数据
for device_id in range(num_devices):
    # 随机选择类别
    random_classes = list(range(num_classes))
    random.shuffle(random_classes)
    selected_classes = random_classes[:selected_classes_per_device]

    # 获取已选类别的索引
    selected_indices = []
    for cls in selected_classes:
        class_indices = np.where(np.array(y_train) == cls)[0]
        selected_indices.extend(class_indices)

    # 在已选类别的索引中进行随机抽样
    random_selected_indices = random.sample(selected_indices, samples_per_device)
    device_data[device_id] = [(x_train[i], y_train[i]) for i in random_selected_indices]
y_train = keras.utils.to_categorical(y_train, num_classes)
# 检查每个设备上的样本数量
for i in range(num_devices):
    print(f"Device {i}: {len(device_data[i])} samples")
    print(f"Device {i}: ", np.unique(np.array([label for _, label in device_data[i]]), return_counts=True))  
data_x = [np.array([sample for sample, _ in device_data[i]]) for i in range(num_devices)]
data_y = [np.array([label for _, label in device_data[i]]) for i in range(num_devices)]
data_y = keras.utils.to_categorical(data_y, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

Device 0: 1152 samples
Device 0:  (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=uint8), array([ 74,  47,  36,  52, 145, 125, 205, 129, 109,  96, 134]))
Device 1: 1152 samples
Device 1:  (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=uint8), array([ 51,  42,  58,  49, 138, 164, 187, 127, 137,  87, 112]))
Device 2: 1152 samples
Device 2:  (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=uint8), array([ 49,  52,  44,  48, 105, 124, 196, 131, 146, 113, 144]))
Device 3: 1152 samples
Device 3:  (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=uint8), array([ 70,  47,  55,  35, 120, 130, 216, 133, 138,  99, 109]))
Device 4: 1152 samples
Device 4:  (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=uint8), array([ 55,  48,  48,  49, 115, 142, 184, 143, 142, 107, 119]))
Device 5: 1152 samples
Device 5:  (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=uint8), array([ 68,  48,  47,  49, 133, 136, 209, 142, 103,  77, 140]))
Device 6: 

In [5]:
def add_partial_backdoor(images, labels_onehot, backdoor_label=8, trigger_size=3, backdoor_ratio=0.5):
    backdoor_images = []
    backdoor_labels_onehot = []

    num_backdoor_samples = int(len(images) * backdoor_ratio)
    backdoor_indices = np.random.choice(len(images), size=num_backdoor_samples, replace=False)

    for idx, (image, label_onehot) in enumerate(zip(images, labels_onehot)):
        backdoor_image = np.copy(image)
        backdoor_label_onehot = np.copy(label_onehot)
        
        if idx in backdoor_indices:
            backdoor_image[-trigger_size:, -trigger_size:] = 1.0
            backdoor_label_onehot = np.zeros_like(label_onehot)
            backdoor_label_onehot[backdoor_label] = 1
            
        backdoor_images.append(backdoor_image)
        backdoor_labels_onehot.append(backdoor_label_onehot)

    return np.array(backdoor_images), np.array(backdoor_labels_onehot)

In [6]:
def  set_model():
    model=Sequential()
        # 定义顺序模型           
    model.add(Convolution2D(
        input_shape = (28,28,1),
        filters = 8,
        kernel_size = 3,
        strides = 1,
        padding = 'same',
        activation = 'relu',
    ))
    # 第一个池化层
    model.add(MaxPooling2D(
        pool_size = 2,
        strides = 2,
        padding = 'same',
    ))
    # 第二个卷积层
    model.add(Convolution2D(16,3,strides=1,padding='same',activation='relu'))
    # 第二个池化层
    model.add(MaxPooling2D(2,2,'same'))                                    
                  
       # 第二个卷积层      
    # 把第三个池化层的输出扁平化为1维
    model.add(layers.Flatten())
    # 第一个全连接层
    model.add(Dense(256,activation='relu'))
    # Dropout
    model.add(Dropout(0.5)) 
    # 第二个全连接层
    model.add(Dense(num_classes,activation='softmax'))

    # 定义优化器
    adam = Adam(lr = 5e-4)        
        
    # 定义优化器,loss function,训练过程中计算准确率
    model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

        #print(model_d[i])
    return model

In [7]:
 def lr_scheduler(s, lr):
    """
    自定义学习率降低函数
    """
    if s == 150:
        lr = 5e-4 

    return lr

lr_callback = LearningRateScheduler(lr_scheduler)

In [8]:
#所有的客户端训练一遍
def fit_allmodel(model_d):
    weights_1 = {}
    print()
    for i in range(len(model_d)):
    # 训练模型
        model_d[i].compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
        model_d[i].fit(data_x[i],data_y[i],batch_size=16,epochs=3,verbose=0,callbacks=[lr_callback])
        #loss,accuracy = model_d[i].evaluate(x_test,y_test,verbose=1)
        
        #print('test loss',loss)
        #print('test accuracy',accuracy)     
    return model_d

In [9]:
def Noise_attack(model_d,t):  
    weights_1 = {}
    for i in range(t):
        weights_1[i] = model_d[i].get_weights() 
        c = len(weights_1[0])
        for j in range(c):
            weights_1[i][j] = skimage.util.random_noise(weights_1[i][j],mode="gaussian",var = 0.2,clip=True)   
    
        model_d[i].set_weights(weights_1[i])
    return model_d

In [10]:
def sc_attack(model_d,t):  
    weights_1 = {}
    for i in range(t):
        weights_1[i] = model_d[i].get_weights() 
        c = len(weights_1[0])
        for j in range(c):
            weights_1[i][j] = 1.5*weights_1[i][j]     
        model_d[i].set_weights(weights_1[i])
    return model_d

In [11]:
def fan_attack(model_d,t):  
    weights_1 = {}
    for i in range(t):
        weights_1[i] = model_d[i].get_weights() 
        c = len(weights_1[0])
        for j in range(c):
            weights_1[i][j] = -weights_1[i][j]
    
        model_d[i].set_weights(weights_1[i])
    return model_d

In [12]:
def fedavg_equal_weights(models):
    """
    models: 每个客户端模型的列表
    """
    num_clients = len(models)
    global_weights = [w / num_clients for w in models[0].get_weights()]

    for model in models[1:]:
        for i, w in enumerate(model.get_weights()):
            global_weights[i] += w / num_clients

    return global_weights


In [13]:
def distribute_global_model(global_model, clients):
    distributed_models = []
    for client in clients:
        client_model = Sequential.from_config(global_model.get_config())
        client_model.set_weights(global_model.get_weights())
        distributed_models.append(client_model)
    return distributed_models

In [14]:
############***************************从这一行开始是防御的函数代码
#*****************第一轮
#先选取百分之50，截取数据
#然后学习字典

#*****************后续
#先选取百分之50，截取数据
#学习字典Dt
#截取数据
#使用last_D，重构
#得到重构误差
#K-means-fedavg

In [15]:
##fir_model_  需要提取百分之50的最相似数据。
def  select_min (model_d):
    weights_1 = {}
    for i in range(len(model_d)):
        weights_1[i] = model_d[i].get_weights()         
    weights_array = np.array(list(weights_1.values()))
    avg = np.mean(weights_array, axis=0)
    n =len(model_d)
    euclidean = [[0 for j in range(len(avg))] for i in range(n)]
    euclidean_array = np.array(euclidean)
    # 计算均值向量
    for j in range (n):
        for i in range(len(avg)):
            a_flat = weights_array[j][i].flatten()
            avg_flat = avg[i].flatten()
            euclidean[j][i] = np.linalg.norm(a_flat - avg_flat)
    # 计算每个数据与均值向量之间的欧氏距离

    row_sums = np.sum(euclidean, axis=1)
    sorted_sums = np.sort(row_sums)

    # 找到前50%最小的值
    top_n = int(n/2)

    min_indices = np.argpartition(row_sums, top_n)[:top_n]
    print(min_indices)
        
    return min_indices,weights_1

In [16]:
####截取数据
def X_dic(weights, min_indices,t):
    
    extracted_vectors = [[0 for j in range(len(min_indices))] for i in range(len(weights[0]))]
    
    for i in range(len(min_indices)):
        ind_x = min_indices[i]
        j = 0        
        for weight in weights[ind_x]:
            # 检查权重是否为int类型           
            weight_flat = weight.flatten()
            num_elements_to_extract = min(t, len(weight_flat))
            extracted = weight_flat[:num_elements_to_extract]
            # 将提取的列向量添加到列表中
            #print(extracted)
            extracted_vectors[j][i] = extracted
            j = j+1                      
    return extracted_vectors
    

In [17]:
import concurrent.futures
import spams

def train_dictionary(X):
    train_params = {
        'K': 15,
        'lambda1': 0.1,
        'numThreads': 2,
        'batchsize': 400,
        'iter': 200,
        'verbose': False
    }
    D = spams.trainDL(X, **train_params)
    return D

def learn_dic(X_all, K):
    dictionary_size = K

    # 创建线程池
    pool = concurrent.futures.ThreadPoolExecutor(max_workers=len(X_all))

    # 存储字典的列表
    D_set = []

    # 循环开始
    futures = []
    for i in range(len(X_all)):
        X_1 = np.array(X_all[i])
        X_1 = X_1.T
        X = np.asfortranarray(X_1, dtype=np.float64)

        future = pool.submit(train_dictionary, X)
        futures.append(future)

    # 等待所有线程完成
    concurrent.futures.wait(futures)

    # 获取所有线程的结果
    for future in futures:
        D = future.result()
        D_set.append(D)

    # 关闭线程池
    pool.shutdown()

    return D_set


In [18]:
######重构数据时需要截取的数据。
def slicing_X(weights,t,n):
    
    extracted_vectors = [[0 for j in range(n)] for i in range(len(weights[0]))]
    
    for i in range(n):
        j = 0        
        for weight in weights[i]:
            # 检查权重是否为int类型           
            weight_flat = weight.flatten()
            num_elements_to_extract = min(t, len(weight_flat))
            extracted = weight_flat[:num_elements_to_extract]
            # 将提取的列向量添加到列表中
            #print(extracted)
            extracted_vectors[j][i] = extracted
            j = j+1                      
    return extracted_vectors

In [19]:
import concurrent.futures
import spams

def lasso_thread(X, D, param_lasso, i, j, reconstruction_error):
    alpha = spams.lasso(X, D=D, return_reg_path=False, **param_lasso)
    X_reconstructed = np.dot(D, alpha.toarray())
    error_n = np.linalg.norm(X[:, j] - X_reconstructed[:, j])
    reconstruction_error[j, i] += error_n
    
def Reconstructed_2(D_all, X_all):
    X_reconstructed = [[0 for j in range(n)] for i in range(len(weights[0]))]
    reconstruction_error = np.zeros((30, 8))

    param_lasso = {
        'lambda1': 0.1,
        'numThreads': 2,
        'mode': 0,
        'pos': True,
        'verbose': False
    }

    # 创建线程池
    pool = concurrent.futures.ThreadPoolExecutor(max_workers=len(D_all))

    # 循环开始
    futures = []
    for i in range(len(D_all)):
        X_1 = np.asfortranarray(X_all[i], dtype=np.float64)
        X = np.asfortranarray(X_1.T)
        D = np.asfortranarray(D_all[i], dtype=np.float64)

        for j in range(n):
            future = pool.submit(lasso_thread, X, D, param_lasso, i, j, reconstruction_error)
            futures.append(future)

    # 等待所有线程完成
    concurrent.futures.wait(futures)

    # 关闭线程池
    pool.shutdown()

    return reconstruction_error


In [20]:
#服务器将客户端数据集中并分配给客户端
def fed_avg_1(model_d): 
    weights = {}
    loss_all=[]
    acc_all=[]
    ##
    #这个for是得到所有模型的参数，那在这里保存所有模型的参数
    for i in range(len(model_d)): 
        weights[i] = model_d[i].get_weights() 
 #################################################
    weights_avg =[]
    weights_sum = weights[n-1].copy()  ###################################这里用了copy就不会出现avg=50的问题了
 ###########################################   
    for i in range (len(weights[0])):  #  1个客户端的n层参数
        for j in range (len (weights)-1): #相当于10个客户端
            weights_sum[i] = weights_sum[i]+ weights[j][i]   #这里出现过错误，应该是先客户端【j】再客户端的层【i】
        weights_sum[i] = weights_sum[i] /len(weights)
        weights_avg.append(weights_sum[i])  
    #将集合后的数据分配给每一个客户端
    for i in range (len (weights)):
        model_d[i].set_weights(weights_avg)  
    #测试客户端的精度
    loss,accuracy= model_d[0].evaluate(x_test,y_test)
#################################################   
    return model_d,loss,accuracy

In [21]:
def fed_avg_2(model_d,reconstruction_error):  
    ###使用K-means
    num_clusters = 2  # 聚类数量
    # 创建并拟合K-means模型
    
    sse = np.sum(reconstruction_error, axis=1)

    # 找到 SSE 最小和最大的行标签
    min_index = np.argmin(sse)
    max_index = np.argmax(sse)
    print(min_index)
    print(max_index)
    init_centers = np.array([reconstruction_error[min_index], reconstruction_error[max_index]]) 
     
    kmeans = KMeans(n_clusters=num_clusters, init=init_centers, n_init=1)
    kmeans.fit(reconstruction_error)
    # 获取聚类结果
    labels = kmeans.labels_
    unique_labels, counts = np.unique(labels, return_counts=True)

    # 找到数量最多的标签
    most_common_label = unique_labels[np.argmax(counts)]

    # 获取数量最多的标签对应的索引
    most_common_label_indices = np.where(labels == most_common_label)[0]
    print(most_common_label_indices)
  #################################################   
    for i in range(len(model_d)): 
        weights[i] = model_d[i].get_weights() 
    weights_avg =[]
    weights_sum = weights[n-1].copy()  
 ###########################################   
    for i in range (len(weights[0])):  #  1个客户端的n层参数
        for j in range (len (most_common_label_indices)-1): #相当于10个客户端
            index = most_common_label_indices[j]
            weights_sum[i] = weights_sum[i]+ weights[index][i]   #这里出现过错误，应该是先客户端【j】再客户端的层【i】
        weights_sum[i] = weights_sum[i] /len(most_common_label_indices)
        weights_avg.append(weights_sum[i])  
    #将集合后的数据分配给每一个客户端
    for i in range (len (weights)):
        model_d[i].set_weights(weights_avg)  
    #测试客户端的精度
    loss,accuracy= model_d[0].evaluate(x_test,y_test)
#################################################   
    return model_d,loss,accuracy
    
    
    
    #存在一个问题，我的攻击都是写在了聚合里，这边要更改，要把聚合写在fit_allmodel里。
    

In [22]:
import skimage 
from sklearn.cluster import KMeans

In [23]:
from keras.models import load_model
from keras.models import clone_model
model_list =[]
model_avg = load_model('my_model_1_9.h5')
loss,accuracy= model_avg.evaluate(x_test,y_test)
for _ in range(30):
    model_copy = clone_model(model_avg)
    model_copy.set_weights(model_avg.get_weights())

    model_list.append(model_copy)


2023-06-11 20:19:24.351862: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2023-06-11 20:19:24.518582: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:86:00.0 name: NVIDIA Tesla V100-PCIE-16GB computeCapability: 7.0
coreClock: 1.38GHz coreCount: 80 deviceMemorySize: 15.78GiB deviceMemoryBandwidth: 836.37GiB/s
2023-06-11 20:19:24.518689: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2023-06-11 20:19:24.524208: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2023-06-11 20:19:24.529192: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcufft.so.10
2023-06-11 20:19:24.529887: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcu

556/556 [==============================] - 6s 11ms/step - loss: 0.6280 - accuracy: 0.8234


In [ ]:

n = int(30)
t = 9

# *****************第一轮
# 先选取百分之50，截取数据
# 然后学习字典
loss_9 = []

# *****************后续
# 先选取百分之50，截取数据
# 学习字典Dt
# 截取数据
# 使用last_D，重构
# 得到重构误差
# K-means-fedavg

model_list = fit_allmodel(model_list)
model_list = sc_attack(model_list, t)
min_indices, weights = select_min(model_list)
extracted_X = X_dic(weights, min_indices, 15)
model_list, loss, accuracy = fed_avg_1(model_list)
D = learn_dic(extracted_X, 15)

### Reconstructed_2，里面有几个参数，需要设置成

for i in range(100):
    # 先训练
    updated_model_list = list(model_list)
    model_list.clear()
    updated_model_list = fit_allmodel(updated_model_list)
    updated_model_list = sc_attack(updated_model_list, t)
    min_indices, weights = select_min(updated_model_list)
    extracted_X = X_dic(weights, min_indices, 15)
    X = slicing_X(weights, 15, n) # 20是截取数据的长度
    # 重构误差使用n-1次迭代的字典重构n次的误差
    reconstruction_error = Reconstructed_2(D, X)
    updated_model_list, loss, accuracy = fed_avg_2(updated_model_list, reconstruction_error)
    D = learn_dic(extracted_X, 15)

    print("***********************************************************************")
    loss_9.append(loss)
    print(i)
    model_list = list(updated_model_list)
    updated_model_list.clear()
    model_list[0].save('my_model_1_9.h5')
scipy.io.savemat('loss_9_1.mat', mdict={'loss_9': loss_9})



[20 16 28 27 26 25 24 29 22  9 10 21 12 13 23]
556/556 [==============================] - 2s 4ms/step - loss: 0.9874 - accuracy: 0.8147

[26 19 13 10 20 21 25 23 16 14 12 27 17 28 22]
29
6
[ 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29]
556/556 [==============================] - 2s 4ms/step - loss: 0.7132 - accuracy: 0.8233
***********************************************************************
0

[13 29 28 25 23 22 21 20 16  9 10 11 14 26 27]
12
3
[ 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29]
556/556 [==============================] - 2s 3ms/step - loss: 0.7074 - accuracy: 0.8244
***********************************************************************
1

[24 21 16 29 22 23 18 20 25 28 12 13 10 19 17]
26
5
[ 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29]
556/556 [==============================] - 2s 3ms/step - loss: 0.7063 - accuracy: 0.8257
***********************************************************************
2

[13 29 28 27 26 23 

In [ ]:
import scipy
from scipy.io import loadmat
scipy.io.savemat('loss_9_1.mat', mdict={'loss_9': loss_9,})
    #批量保存data

In [ ]:
model_list =[]

for i in range(30):
    model = set_model()
    model_list.append(model)
n = int(30)
t = 6
#*****************第一轮
#先选取百分之50，截取数据
#然后学习字典
loss_6 = []
#*****************后续
#先选取百分之50，截取数据
#学习字典Dt
#截取数据
#使用last_D，重构
#得到重构误差
#K-means-fedavg

model_list = fit_allmodel(model_list)
model_list = sc_attack(model_list,t)
min_indices,weights = select_min(model_list)
extracted_X = X_dic(weights, min_indices,15)
model_list,loss,accuracy = fed_avg_1(model_list)
D = learn_dic(extracted_X,15)
###   Reconstructed_2,里面有几个参数，需要设置成

for i in range (100):   
    #先训练
    model_list = fit_allmodel(model_list)
    model_list = sc_attack(model_list,t)
    min_indices,weights = select_min(model_list)
    extracted_X = X_dic(weights, min_indices,15)
    X = slicing_X(weights,15,n) #20是截取数据的长度
    #重构误差 使用n-1次迭代的字典重构n次的误差
    reconstruction_error = Reconstructed_2(D,X)
    model_list,loss,accuracy = fed_avg_2(model_list,reconstruction_error)
    D = learn_dic(extracted_X,15)    
    print("***********************************************************************")
    loss_6.append(loss)
    print(i)
scipy.io.savemat('loss_6_1.mat', mdict={'loss_6': loss_6,})

In [ ]:
model_list =[]

for i in range(30):
    model = set_model()
    model_list.append(model)
n = int(30)
t = 3
#*****************第一轮
#先选取百分之50，截取数据
#然后学习字典
loss_3 = []
#*****************后续
#先选取百分之50，截取数据
#学习字典Dt
#截取数据
#使用last_D，重构
#得到重构误差
#K-means-fedavg

model_list = fit_allmodel(model_list)
model_list = sc_attack(model_list,t)
min_indices,weights = select_min(model_list)
extracted_X = X_dic(weights, min_indices,15)
model_list,loss,accuracy = fed_avg_1(model_list)
D = learn_dic(extracted_X,15)
###   Reconstructed_2,里面有几个参数，需要设置成

for i in range (100):   
    #先训练
    model_list = fit_allmodel(model_list)
    model_list = sc_attack(model_list,t)
    min_indices,weights = select_min(model_list)
    extracted_X = X_dic(weights, min_indices,15)
    X = slicing_X(weights,15,n) #20是截取数据的长度
    #重构误差 使用n-1次迭代的字典重构n次的误差
    reconstruction_error = Reconstructed_2(D,X)
    model_list,loss,accuracy = fed_avg_2(model_list,reconstruction_error)
    D = learn_dic(extracted_X,15)    
    print(i)
    print("***********************************************************************")
    loss_3.append(loss)
scipy.io.savemat('loss_3_1.mat', mdict={'loss_3': loss_3,})